In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.options import Options
from datetime import datetime

In [2]:
month = "January"
month_abbr = "Jan"
month_num = "01"

## iStructE Events

In [3]:
import time

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

chrome_options = Options()  
chrome_options.add_argument("--headless")

driver = webdriver.Chrome("C:/Program Files (x86)/Google/chromedriver.exe", options=chrome_options)

title_istructe = []
body_1_istructe = []
body_2_istructe = []
date_istructe = []
time_istructe = []
link_istructe = []

#Looping through the first 2 pages of the iStructE events site
for i in range(1,3):
    istructe = "https://www.istructe.org/events/search/?from=&lat=&lng=&maxRadius=50&p=" + str(i) + "&s=4&searchTerm=&to="
    driver.get(istructe)
    time.sleep(5)  
    soup = BeautifulSoup(driver.page_source)
    for j in soup.find_all(class_='card-promo__description'):
        date_ = j.find('div', attrs = {'class': 'card-promo__details'}).text
        if month in str(date_):
            for k in j.find_all('a', href=True):
                link = k['href']
                istructe_start = "https://www.istructe.org"
                event = istructe_start + link
                URL = str(event)
                link_istructe.append(URL)
                
                # Opening individual event link
                r = requests.get(URL) 
                soup = BeautifulSoup(r.content, 'html5lib')
                
                # Title
                title = soup.find('h5', attrs = {'class': 'card-summary__title'})
                title = title.text
                title_istructe.append(title)
                
                #Description
                body_1 = soup.find('div', attrs = {'class':'body-copy'})
                body_1 = body_1.text.strip()
                body_1_istructe.append(body_1)
                
                #Date
                date_time = soup.find('p').text
                date = date_time.split("2021",1)[0]
                date_istructe.append(date)
                
                #Time
                time_event = date_time.split("2021",1)[1]
                time_istructe.append(time_event)
                
                #Additional Details
                body_2 = soup.find_all('div', attrs = {'class': 'body-copy'})[1]
                if body_2.p is None:
                    body_2_istructe.append('')
                else:
                    body_2 = body_2.p.text
                    body_2 = body_2.replace("\n\n"," ")
                    body_2_istructe.append(body_2)
                    
istructe_df = pd.DataFrame(list(zip(link_istructe, title_istructe, date_istructe, time_istructe, body_1_istructe, body_2_istructe)), 
               columns =['Link', 'Title', 'Date', 'Time', 'Description', 'Additional Details']) 

## CIBSE events

In [4]:
from bs4 import BeautifulSoup
import requests

links_cibse = []
title_cibse = []
body_cibse = []
date_cibse = []
time_cibse = []
description_cibse = []

# Only looping through the first four pages of the events pages (assume that all of the month's events will be within the first four pages)
for k in range(1,4):

    URL = "https://www.cibse.org/training-events?page=" + str(k) + "&f=events"
    r = requests.get(URL) 

    soup = BeautifulSoup(r.content, 'html5lib')
    for i in soup.find_all('div', attrs = {'class': 'large-6 columns info'}):
        
        # Obtaining all the text in each 'div' class with 'large-6 columns info'
        info = i.text
        
        # Month defined at the beginning
        if month in str(info):
            
            # Extracting all links
            links = i.a
            links = links['href']
            event_link = "https://www.cibse.org" + links
            
            # Entering the link of the event
            r_event = requests.get(event_link)
            soup_event = BeautifulSoup(r_event.content, 'html5lib')
            
            if 'CIBSE - Page not found' in soup_event.find('body').text:
                continue
                
            else:
                links_cibse.append(event_link)
                
                # Extracting date and time
                date_time = soup_event.find('div', attrs = {'class': 'info'}).text
                date = date_time.split("2021",1)[0]
                date_cibse.append(date.strip())
                time = date_time.split("2021",1)[1]
                time_cibse.append(time.strip().partition("\n")[0])

                # Extracting title
                title = soup_event.find('h1').text
                title_cibse.append(title)

                # Extracting description
                description = soup_event.find('div', attrs = {'class': 'large-8 columns padded-sides body-copy pull-2 float-right'})

                description_per_event = []
                for k in description.find_all('strong'):
                    if k.text != '':
                        description_per_event.append(k.text)
                description_per_event = ' '.join(description_per_event)
                description_cibse.append(description_per_event)

                further_description_per_event = []
                for j in description.find_all('p'):
                    if not j.find_parent('div', attrs = {'class': 'spacer content-block red event-book'}):
                        if j.text != '':
                            further_description_per_event.append(j.text)
                further_description_per_event = ' '.join(further_description_per_event)
                body_cibse.append(further_description_per_event)

cibse_df = pd.DataFrame(list(zip(links_cibse, title_cibse, date_cibse, time_cibse, description_cibse, body_cibse)), \
                       columns = ['Link', 'Title', 'Date', 'Time', 'Description', 'Additional Details'])

## APM events

In [5]:
import time
import re

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

links_apm = []
title_apm = []
body_apm = []
date_apm = []
time_apm = []
description_apm = []

chrome_options = Options()  
chrome_options.add_argument("--headless")

driver = webdriver.Chrome("C:/Program Files (x86)/Google/chromedriver.exe", options=chrome_options)
    
URL = "https://www.apm.org.uk/event/"
r = requests.get(URL) 

driver.get(URL)
time.sleep(5)  
soup = BeautifulSoup(driver.page_source)

# Extracting all links in particular month
for i in soup.find('div', attrs = {'class': 'table-responsive'}).find_all(text=re.compile("%s"%month_abbr)):
    event_link = i.findParent('td').findPrevious('td').find(href = True)
    event_link = "http://www.apm.org.uk" + event_link['href']
    links_apm.append(event_link)
    
    r_event = requests.get(event_link)
    soup_event = BeautifulSoup(r_event.content, 'html5lib')
    
    #Extract title and description
    title_apm.append(soup_event.find('div', attrs = {'class': 'col-md-8'}).find('h2').text)
    description = soup_event.find('div', attrs = {'class': 'col-md-8'}).find(['h2', 'p'])
    description_apm.append(description.text)
    
    body = description.findAllNext(['p','ul'], text = True)
    body = [i.text for i in body]
    body = ' '.join(body)
    body_apm.append(body)
    
    #Extracting date and time
    datetime = soup_event.find('div', attrs = {'class': 'col-md-4'}).find('dl')
    datetime = [i for i in datetime]
    datetime = [i for i in datetime if (i.string != None)]
    date = [i for i in datetime if (str(month) in i.string)]
    date = date[0].text.replace(' 2021', '')
    date_apm.append(date)
    
    time_event = [inx for inx, val in enumerate(datetime) if ('Time' in val.string)]
    time_apm.append(datetime[time_event[0] + 2].text)
    
apm_df = pd.DataFrame(list(zip(links_apm, title_apm, date_apm, time_apm, description_apm, body_apm)), \
                       columns = ['Link', 'Title', 'Date', 'Time', 'Description', 'Additional Details'])

## CIRCA events

In [6]:
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

links_ciria = []
title_ciria = []
body_ciria = []
date_ciria = []
time_ciria = []
description_ciria = []

chrome_options = Options()  
chrome_options.add_argument("--headless")

driver = webdriver.Chrome("C:/Program Files (x86)/Google/chromedriver.exe", options=chrome_options)
    
URL = "https://www.ciria.org/CIRIA/Events/Events_overview/Events/Events_overview.aspx?hkey=94fbc810-200b-4650-8d0d-863d30607512"
r = requests.get(URL) 

driver.get(URL)
time.sleep(5)  
soup = BeautifulSoup(driver.page_source)

#Extract links to all events in month
for i in soup.find('div', attrs = {'id': 'ctl01_TemplateBody_WebPartManager1_gwpciNewQueryMenuCommon_ciNewQueryMenuCommon_ResultsGrid_GridPanel1'}).find('tbody').find_all(lambda tag:tag.name=="td" and str(month_num) in tag.string):
    event_link = i.findNextSibling('td').find('a', href = True)
    event_link = event_link['href']
    links_ciria.append(event_link)
    
    r_event = requests.get(event_link)
    driver.get(event_link)
    time.sleep(5)
    soup_event = BeautifulSoup(driver.page_source)
    
    #Extract all titles and descriptions
    title = soup_event.find('div', attrs = {'class': 'TitleBarTitle'}).text
    title_ciria.append(title.strip())

    description_ciria.append(soup_event.find('div', attrs = {'id': 'ctl01_TemplateBody_WebPartManager1_gwpciEventDisplay_ciEventDisplay_DescriptionDiv'}).find('span').text)
    
    #Extract all date and times
    datetime = soup_event.find('div', attrs = {'class': 'Description'}).find(lambda tag:tag.name == 'strong' and tag.string == "When").text
    
    next_datetime = soup_event.find('div', attrs = {'class': 'Description'}).find(lambda tag:tag.string == "When").findNextSibling('strong').text
    
    # Separating out the date and time into individual lists
    for j in soup_event.find('div', attrs = {'class': 'Description'}).find_all('p'):
        if "When" in j.text:
            date_time = j.text[j.text.find(datetime)+len(datetime):j.text.rfind(next_datetime)]
            date_ciria.append([i for i in date_time.split('\n') if (i != '')][0].replace(' 2021', ''))
            time_ciria.append([i for i in date_time.split('\n') if (i != '')][1])
    
ciria_df = pd.DataFrame(list(zip(links_ciria, title_ciria, date_ciria, time_cibse, description_ciria)), \
                       columns = ['Link', 'Title', 'Date', 'Time', 'Description'])

## CICES events

In [7]:
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

links_cices = []
title_cices = []
date_cices = []
time_cices = []
description_cices = []

chrome_options = Options()  
chrome_options.add_argument("--headless")

driver = webdriver.Chrome("C:/Program Files (x86)/Google/chromedriver.exe", options=chrome_options)
    
URL = "https://www.cices.org/news/events/"
r = requests.get(URL) 

driver.get(URL)
time.sleep(5)  
soup = BeautifulSoup(driver.page_source)

#Extracting all data
for i in soup.find_all('a', attrs = {'class': 'box event'}):
    if str(month_num) in i.attrs["data-date"]:
        event_link = i['href']
        links_cices.append(event_link)
        
        r_event = requests.get(event_link)
        driver.get(event_link)
        time.sleep(5)
        soup_event = BeautifulSoup(driver.page_source)
        
        title = soup_event.find('div', attrs = {'class': 'heading-top'}).text
        title_cices.append([i for i in title.split('\n') if (i != '')][0])
        
        description = soup_event.find('div', attrs = {'class': 'section section-event'}).find_all('p')
        
        description = [i.text for i in description]
        description = ' '.join(description)
        description = description.translate(str.maketrans({'\n': ' ', '\t': '', '\xa0': ' '}))
        description_cices.append(description)
        
        datetime = soup_event.find('div', attrs = {'class': 'column'})
        
        date_cices.append(datetime.find('h2').text)
        
        time_cices.append(datetime.find('p').text.replace('\t','').replace('Starts: ','').replace('Ends:', ' -'))
        
cices_df = pd.DataFrame(list(zip(links_cices, title_cices, date_cices, time_cices, description_cices)), \
                       columns = ['Link', 'Title', 'Date', 'Time', 'Description'])

## CIHT events

In [8]:
import time

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

chrome_options = Options()  
chrome_options.add_argument("--headless")

driver = webdriver.Chrome("C:/Program Files (x86)/Google/chromedriver.exe", options=chrome_options)

title_ciht = []
description_ciht = []
date_ciht = []
time_ciht = []
link_ciht = []
next_month = False
page_no = 1

# If the month shown on the event is not the month interested, do not go to next page
# Extract all links
while next_month == False:
    ciht = "https://www.ciht.org.uk/search-results/?events=1&search=#/?page=" + str(page_no) +"&filters=%7B%22Region%22:%22%22,%22ContentType%22:%22%22,%22Interests%22:%22%22,%22Upcoming%22:%22True%22,%22Current%22:%22True%22,%22Past%22:%22False%22%7D"
    driver.get(ciht)
    time.sleep(5)  
    soup = BeautifulSoup(driver.page_source)
    
    for i in soup.find('div', attrs = {'ng-show': 'stateEvents'}).find_all('div', attrs = {'class': 'item ng-scope'}):
        if month in i.text:
            event_link = "https://www.ciht.org.uk/" + i.find('a')['href']
            link_ciht.append(event_link)

        else:
            next_month = True
        
    page_no += 1

# Extract all other information
for i in link_ciht:
    r_event = requests.get(i)
    driver.get(i)
    time.sleep(5)
    soup_event = BeautifulSoup(driver.page_source)
    
    details = soup_event.find('div', attrs = {'class': 'col-md-9'})
    datetime = details.find('strong').text
    datetime = datetime.strip().split(',')
    date_ciht.append(datetime[0].replace(' 2021', ''))
    time_ciht.append(datetime[1].strip())
    
    title_ciht.append(details.find('h1').text)
    
    for j in details.find_all('p', recursive = False):
        if j.text.strip() != "":
            description_ciht.append(j.text.strip().replace("\xa0", " "))
            
ciht_df = pd.DataFrame(list(zip(link_ciht, title_ciht, date_ciht, time_ciht, description_ciht)), \
                       columns = ['Link', 'Title', 'Date', 'Time', 'Description'])

## IChemE events

In [9]:
import time

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

chrome_options = Options()  
chrome_options.add_argument("--headless")

driver = webdriver.Chrome("C:/Program Files (x86)/Google/chromedriver.exe", options=chrome_options)

title_icheme = []
description_icheme = []
date_icheme = []
time_icheme = []
link_icheme = []
next_month = False
page_no = 1

# If the month shown on the event is not the month interested, do not go to next page
# Extract all links
while next_month == False:
    icheme = "https://www.icheme.org/membership/communities/all-upcoming-events/page-" + str(page_no) +"/"
    driver.get(icheme)
    time.sleep(5)  
    soup = BeautifulSoup(driver.page_source)
    
    event = soup.find('div', attrs = {'class': 'event-list-panel-wrap event-list-panel-wrap--m-top'})
    
    for i in event.find_all('a', href = True):
        event_link = i['href']
    
        if (month in i.text) and ("Online" in i.text):
            link_icheme.append("https://www.icheme.org/" + event_link)

        elif not (month in i.text):
            next_month = True
        
    page_no += 1
    
# Extract all other information
for i in link_icheme:
    r_event = requests.get(i)
    driver.get(i)
    time.sleep(5)
    soup_event = BeautifulSoup(driver.page_source)
    
    title_icheme.append(soup_event.find('h1', attrs = {'class': 'section-title'}).text)
    datetime = soup_event.find('ul', attrs = {'class':'event-overview'}).text.split('\n')
    date_icheme.append(datetime[datetime.index('Date From') + 1].replace(' 2021', ''))
    the_time = datetime[datetime.index('Location') + 1]
    time_icheme.append(re.findall('[0-9][0-9]:[0-9][0-9] [A-Z]{3}', the_time)[0])
    
    detail = soup_event.find('div', attrs = {'class':'umb-body umb-body--m-bottom'}).find('p').text
    description_icheme.append(detail)

icheme_df = pd.DataFrame(list(zip(link_icheme, title_icheme, date_icheme, time_icheme, description_icheme)), \
                       columns = ['Link', 'Title', 'Date', 'Time', 'Description'])

In [12]:
# Merge and extract to Excel
df = [istructe_df, cibse_df, apm_df, ciria_df,cices_df,ciht_df,icheme_df]
df = pd.concat(df)

df_compiled = pd.DataFrame()
df_compiled['Date and Time'] = df[df.columns[2:4]].apply(lambda x: ','.join(x.dropna()), axis = 1)
df_compiled['Title'] = df['Title']
df_compiled['Description'] = df.iloc[:,[4,0]].apply(lambda x: ','.join(x.dropna()), axis = 1)

df_compiled.to_excel('Events.xlsx', index = False)